In [ ]:
from xd import *
from utils import *
import esutil, yaml, sys, os, argparse
import healpy as hp
#from systematics import *
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import numpy as np
import astropy.io.fits as fits

from cmass_modules import io
from utils import matchCatalogsbyPosition, hpHEALPixelToRaDec, HealPixifyCatalogs, spatialcheck
from xd import mixing_color, XD_fitting_X, assignCMASSProb, doVisualization_1d
from run_DMASS import priorCut_test

rootdir = '/global/cscratch1/sd/bwarner/'
cmass = esutil.io.read(rootdir + 'cmass_in_st82.fits')
train_sample = esutil.io.read(rootdir + 'train_sample_des.fits')
#cmass = esutil.io.read(rootdir + 'cmass_in_st82.fits')

In [2]:
#gold_st82 = io.SearchAndCallFits(path = '/n/des/lee.5922/data/gold_cat/', keyword='Y1A1_GOLD_STRIPE82_v2_000')
#gold_st82 = clean_gold(gold_st82)

#use gold catalog used for matching for now

# calling stripe82 gold catalogue -----------------------------
train_path = '/global/cscratch1/sd/bwarner/gold_cat/GOLD_STRIPE82/'
path =  train_path
train_keyword = 'Y1A1_GOLD_STRIPE82_v2'
    
columns = ['FLAGS_GOLD', 'FLAGS_BADREGION', 'MAG_MODEL_G', 'MAG_MODEL_R', 'MAG_MODEL_I', 'MAG_MODEL_Z',\
               'MAG_DETMODEL_G', 'MAG_DETMODEL_R', 'MAG_DETMODEL_I', 'MAG_DETMODEL_Z', 'MAGERR_DETMODEL_G',\
               'MAGERR_DETMODEL_R', 'MAGERR_DETMODEL_I', 'MAGERR_DETMODEL_Z', 'MAGERR_MODEL_G', 'MAGERR_MODEL_R',\
               'MAGERR_MODEL_I', 'MAGERR_MODEL_Z', 'MAG_AUTO_G', 'MAG_AUTO_R', 'MAG_AUTO_I', 'MAG_AUTO_Z', 'RA',\
               'DEC', 'COADD_OBJECTS_ID', 'MODEST_CLASS', 'HPIX', 'DESDM_ZP',\
               'SLR_SHIFT_G', 'SLR_SHIFT_R', 'SLR_SHIFT_I', 'SLR_SHIFT_Z', 'SLR_SHIFT_Y', 'EBV']

gold_st82 = io.SearchAndCallFits(path = train_path, columns = columns, keyword = train_keyword )
gold_st82 = gold_st82[(gold_st82['MODEST_CLASS'] == 1)&(gold_st82['FLAGS_GOLD'] == 0 )]
    
def keepGoodRegion(des, hpInd = False, balrog=None):
    import healpy as hp
    import fitsio
    # 25 is the faintest object detected by DES
    # objects larger than 25 considered as Noise
    
    path = '/global/cscratch1/sd/bwarner/'
    #LSSGoldmask = fitsio.read(path+'Y1LSSmask_v2_il22_seeil4.0_nside4096ring_redlimcut.fits')
    #LSSGoldmask = fitsio.read(path+'Y1LSSmask_v1_il22seeil4.04096ring_redlimcut.fits')
    LSSGoldmask = fitsio.read(path+'Y1LSSmask_v2_redlimcut_il22_seeil4.0_4096ring.fits')
    #Y1LSSmask_v1_il22seeil4.04096ring_redlimcut.fits
    #frac_cut = LSSGoldmask['FRAC'] > 0.8
    #ind_good_ring = LSSGoldmask['PIXEL'][frac_cut]
    ind_good_ring = LSSGoldmask['PIXEL']
    
    # healpixify the catalog.
    nside=4096
    # Convert silly ra/dec to silly HP angular coordinates.
    phi = des['RA'] * np.pi / 180.0
    theta = ( 90.0 - des['DEC'] ) * np.pi/180.0

    hpInd = hp.ang2pix(nside,theta,phi,nest=False)
    keep = np.in1d(hpInd, ind_good_ring)
    des = des[keep]
    if hpInd is True:
        return ind_good_ring
    else:
        return des

gold_st82 = keepGoodRegion(gold_st82)

def RemovingSLRReddening(cat):


    if 'SLR_SHIFT_G' not in cat.dtype.names : 
        import numpy.lib.recfunctions as rf   
        #from suchyta_utils.y1a1_slr_shiftmap import SLRShift
        from y1a1_slr_shiftmap import SLRShift
        zpfile = '/n/des/lee.5922/data/systematic_maps/y1a1_wide_slr_wavg_zpshift2.fit'
        slrshift = SLRShift(zpfile, fill_periphery=True)
        offsets_g = slrshift.get_zeropoint_offset('g',cat['RA'],cat['DEC'],interpolate=True)
        offsets_r = slrshift.get_zeropoint_offset('r',cat['RA'],cat['DEC'],interpolate=True)
        offsets_i = slrshift.get_zeropoint_offset('i',cat['RA'],cat['DEC'],interpolate=True)
        offsets_z = slrshift.get_zeropoint_offset('z',cat['RA'],cat['DEC'],interpolate=True)

        offsets = [ offsets_g, offsets_r, offsets_i, offsets_z  ]
        from pandas import DataFrame, concat
        nametag = ['SLR_SHIFT_'+f for f in ['G', 'R', 'I', 'Z'] ]
        catnametag = cat.dtype.names
        try : 
            offsetsdf = DataFrame( offsets, index = nametag ).T
            cat = DataFrame(cat, index = catnametag)
            #del cat['index']
            print('concatenate two ndarrays')
            cat = concat([cat, offsetsdf], axis=1)
            print('dataframe to recordarray')
            cat = cat.to_records()
        
        except ValueError :
            print("Big-endian buffer not supported on little-endian compiler")
            print("Doing byte swapping")
            
            #offsetsdf = np.array(offsetsdf).byteswap().newbyteorder()
            cat = np.array(cat).byteswap().newbyteorder()
            offsetsdf = DataFrame( offsets, index = nametag ).T
            cat = DataFrame(cat)

            print('concatenate two ndarrays')
            cat = concat([cat, offsetsdf], axis=1)
            print('dataframe to recordarray')
            cat = cat.to_records()
            cat.dtype.names = [str(x) for x in cat.dtype.names]
            
            #matched = pd.merge(desData, goldData, on=key, how=how, suffixes = suffixes, left_index=left_index)


    print('Removing SLR Shift ')
    for mag in ['MAG_MODEL', 'MAG_DETMODEL', 'MAG_AUTO']:
        print('  removing SLR from ', mag)
        for b in ['G', 'R', 'I', 'Z']:
            cat[mag + '_'+b] = cat[mag + '_'+b] - cat['SLR_SHIFT'+ '_'+b]

    """
    cat = rf.append_fields(cat, 'SLR_SHIFT_G', offsets_g)
    cat = rf.append_fields(cat, 'SLR_SHIFT_R', offsets_r)
    cat = rf.append_fields(cat, 'SLR_SHIFT_I', offsets_i)
    cat = rf.append_fields(cat, 'SLR_SHIFT_Z', offsets_z)
    """
    return cat


def AddingSFD98Reddening(cat, kind='SPT', coeff = [3.186,2.140,1.569,1.196 ] ):
    import numpy.lib.recfunctions as rf
    import pandas as pd

    band = ['G', 'R', 'I', 'Z']

    if 'EBV' not in cat.dtype.names :   
     
        print('Using SFD98 nside 4096 healpix map')
        print('Bands :',  band)
        #print 'NSIDE = 4096'
        print('coefficients = ', coeff)
        nside = 4096

        #from suchyta_utils.y1a1_slr_shiftmap import SLRShift
        #sfdfile = '/n/des/lee.5922/data/systematic_maps/y1a1_wide_slr_wavg_zpshift2.fit'
        mapname = '/n/des/lee.5922/data/systematic_maps/ebv_sfd98_fullres_nside_4096_nest_equatorial.fits'
        #mapname = '/n/des/lee.5922/data/systematic_maps/ebv_lenz17_nside_4096_nest_equatorial.fits'
        reddening_ring = hp.read_map(mapname)
        hpIndices = np.arange(reddening_ring.size)
        #goodmask = hp.mask_good(reddening_ring)
        #goldmask = 

        goodIndices = hpIndices #hpIndices[goodmask]
        clean_map = reddening_ring #reddening_ring[goodmask]

        sysMap = np.zeros((clean_map.size, ), dtype=[('PIXEL', 'i4'), ('EBV', 'f8'), ('RA', 'f8'), ('DEC', 'f8')])
        sysMap['PIXEL'] = goodIndices
        sysMap['EBV'] = clean_map
        
        sys_ra, sys_dec = hpHEALPixelToRaDec(goodIndices, nside = nside)
        sysMap['RA'] = sys_ra
        sysMap['DEC'] = sys_dec

        from cmass_modules.Cuts import keepGoodRegion
        sysMap = keepGoodRegion(sysMap)
        if kind is 'STRIPE82': sysMap = sysMap[sysMap['DEC'] > -30]
        elif kind is 'SPT': sysMap = sysMap[sysMap['DEC'] < -30]


        cat_hp = cat
        hpind = hpRaDecToHEALPixel(cat_hp['RA'], cat_hp['DEC'], nside= 4096, nest= False)
        #cat_hp.dtype.names = [str(x) for x in cat_hp.dtype.names]
        cat_hp = changeColumnName(cat_hp, name = 'HPIX', rename = 'PIXEL')
        cat_hp['PIXEL'] = hpind
        
        #sfdmap = changeColumnName( sysMap_ge, name = 'SIGNAL', rename = 'SFD98' )


        try : 

            cat_Data = pd.DataFrame(cat_hp)
            sfdData = pd.DataFrame(sysMap)
            matched = pd.merge(cat_Data, sfdData, on='PIXEL', how='left', 
                               suffixes = ['','_sys'], left_index=False, right_index=False)
        except ValueError :
            print("Big-endian buffer not supported on little-endian compiler")
            print("Doing byte swapping ....")

            cat_hp = np.array(cat_hp).byteswap().newbyteorder()
            #sfdmap = np.array(sfdmap).byteswap().newbyteorder()
            cat_Data = pd.DataFrame(cat_hp)
            sfdData = pd.DataFrame(sysMap)
            

            #print cat_Data.keys()
            #print sfdData.keys()
            matched = pd.merge(cat_Data, sfdData, on='PIXEL', how='left', 
                               suffixes = ['','_sys'], left_index=False, right_index=False)
            
        matched_arr = matched.to_records(index=False)
        matched_arr.dtype.names = [str(x) for x in matched_arr.dtype.names]


    else : matched_arr = cat

    print('Adding SFD98 Shift ')
    print('Bands :',  band)
    print('coefficients = ', coeff)

    for mag in ['MAG_MODEL', 'MAG_DETMODEL', 'MAG_AUTO']:
        print('  Adding SFD to ', mag)
        for i,b in enumerate(band):
            matched_arr[mag + '_'+b] = matched_arr[mag + '_'+b] - matched_arr['EBV'] * coeff[i]    

    return matched_arr

params={'SFD98':True}
if 'SFD98' in params : 
    if params['SFD98'] : 
        print('change reddening corrections from SLR to SFD98')
        gold_st82 = RemovingSLRReddening(gold_st82)
        gold_st82 = AddingSFD98Reddening(gold_st82, kind='STRIPE82')

# flags and color cut
mask_all = priorCut_test(gold_st82)
gold_st82 = gold_st82[mask_all]
    
print(gold_st82.size)

##gold_st82 = io.SearchAndCallFits(path = '/global/cscratch1/sd/bwarner/', keyword='gold_st82_prob.fits')


Calling catalogs >>>
Y1A1_GOLD_STRIPE82_v2_000006.fits
Y1A1_GOLD_STRIPE82_v2_000007.fits
Y1A1_GOLD_STRIPE82_v2_000001.fits
Y1A1_GOLD_STRIPE82_v2_000003.fits
Y1A1_GOLD_STRIPE82_v2_000008.fits
Y1A1_GOLD_STRIPE82_v2_000002.fits
Y1A1_GOLD_STRIPE82_v2_000004.fits
Y1A1_GOLD_STRIPE82_v2_000005.fits
reading 1/8 /global/cscratch1/sd/bwarner/gold_cat/GOLD_STRIPE82/Y1A1_GOLD_STRIPE82_v2_000006.fits
reading 2/8 /global/cscratch1/sd/bwarner/gold_cat/GOLD_STRIPE82/Y1A1_GOLD_STRIPE82_v2_000007.fits
reading 3/8 /global/cscratch1/sd/bwarner/gold_cat/GOLD_STRIPE82/Y1A1_GOLD_STRIPE82_v2_000001.fits
reading 4/8 /global/cscratch1/sd/bwarner/gold_cat/GOLD_STRIPE82/Y1A1_GOLD_STRIPE82_v2_000003.fits
reading 5/8 /global/cscratch1/sd/bwarner/gold_cat/GOLD_STRIPE82/Y1A1_GOLD_STRIPE82_v2_000008.fits
reading 6/8 /global/cscratch1/sd/bwarner/gold_cat/GOLD_STRIPE82/Y1A1_GOLD_STRIPE82_v2_000002.fits
reading 7/8 /global/cscratch1/sd/bwarner/gold_cat/GOLD_STRIPE82/Y1A1_GOLD_STRIPE82_v2_000004.fits
reading 8/8 /global/

<ipython-input-2-71694c920e4f>:53: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  if 'SFD98' in params :


CHECK input catalog has only galaxies


In [5]:
#cmass_spec = esutil.io.read('/n/des/lee.5922/data/cmass_cat/galaxy_DR11v1_CMASS_South.fits.gz')
cmass_photo = esutil.io.read('/global/cscratch1/sd/bwarner/galaxy_DR11v1_CMASS_South-photoObj.fits.gz')
train_sample = esutil.io.read('/global/cscratch1/sd/bwarner/galaxy_DR11v1_CMASS_South-photoObj.fits.gz')
print('total num of train', train_sample.size)
print('\n--------------------------------\n applying DES veto mask to CMASS\n--------------------------------')   
train_sample = keepGoodRegion(train_sample)
#fitsio.write( output_dir+'/cmass_in_st82.fits', train_sample)

print('num of train_sample after des veto', train_sample.size)
cmass_stripe = keepGoodRegion(cmass_photo)

#gold_train, cmass_train = matchCatalogsbyPosition( gold , cmass_stripe)
cmass_train=cmass_stripe
gold_train = train_sample.copy()

total num of train 188582

--------------------------------
 applying DES veto mask to CMASS
--------------------------------
num of train_sample after des veto 11368


In [7]:
print(gold_train.dtype)

[('OBJID', '<U19'), ('PARENTID', '<U19'), ('FIELDID', '<U19'), ('SKYVERSION', 'u1'), ('MODE', 'u1'), ('CLEAN', 'u1'), ('RUN', '>i2'), ('RERUN', '<U3'), ('CAMCOL', 'u1'), ('FIELD', '>i2'), ('ID', '>i2'), ('PARENT', '>i2'), ('NCHILD', '>i2'), ('OBJC_TYPE', '>i4'), ('OBJC_PROB_PSF', '>f4'), ('OBJC_FLAGS', '>i4'), ('OBJC_FLAGS2', '>i4'), ('OBJC_ROWC', '>f4'), ('OBJC_ROWCERR', '>f4'), ('OBJC_COLC', '>f4'), ('OBJC_COLCERR', '>f4'), ('ROWVDEG', '>f4'), ('ROWVDEGERR', '>f4'), ('COLVDEG', '>f4'), ('COLVDEGERR', '>f4'), ('ROWC', '>f4', (5,)), ('ROWCERR', '>f4', (5,)), ('COLC', '>f4', (5,)), ('COLCERR', '>f4', (5,)), ('PETROTHETA', '>f4', (5,)), ('PETROTHETAERR', '>f4', (5,)), ('PETROTH50', '>f4', (5,)), ('PETROTH50ERR', '>f4', (5,)), ('PETROTH90', '>f4', (5,)), ('PETROTH90ERR', '>f4', (5,)), ('Q', '>f4', (5,)), ('QERR', '>f4', (5,)), ('U', '>f4', (5,)), ('UERR', '>f4', (5,)), ('M_E1', '>f4', (5,)), ('M_E2', '>f4', (5,)), ('M_E1E1ERR', '>f4', (5,)), ('M_E1E2ERR', '>f4', (5,)), ('M_E2E2ERR', '>f4'

In [6]:
#magnitudes w/ extinction correction: des vs cmass: wiki gold
#sdss cmass catalogs-- google-- Su question

g_sdss = cmass_train['MODELMAG'][:,1]-cmass_train['EXTINCTION'][:,1]
r_sdss = cmass_train['MODELMAG'][:,2]-cmass_train['EXTINCTION'][:,2]
i_sdss = cmass_train['MODELMAG'][:,3]-cmass_train['EXTINCTION'][:,3]
gr_sdss = g_sdss - r_sdss
ri_sdss = r_sdss - i_sdss

gr_des = gold_train['MAG_DETMODEL_G']- gold_train['MAG_DETMODEL_R']
ri_des = gold_train['MAG_DETMODEL_R']- gold_train['MAG_DETMODEL_I']

gr_gold = gold_st82['MAG_DETMODEL_G']- gold_st82['MAG_DETMODEL_R']
ri_gold = gold_st82['MAG_DETMODEL_R']- gold_st82['MAG_DETMODEL_I']

ValueError: no field of name MAG_DETMODEL_G

In [ ]:
x = np.linspace(-2,4,10)
dperp = 0.55 + x/8.

In [ ]:
gr_sdss_err = np.sqrt(cmass_train['MODELMAGERR'][:,1]**2 + cmass_train['MODELMAGERR'][:,2]**2)
ri_sdss_err = np.sqrt(cmass_train['MODELMAGERR'][:,2]**2 + cmass_train['MODELMAGERR'][:,3]**2)

gr_des_err = np.sqrt(gold_train['MAGERR_DETMODEL_G']**2 +  gold_train['MAGERR_DETMODEL_R']**2 )
ri_des_err = np.sqrt(gold_train['MAGERR_DETMODEL_R']**2 +  gold_train['MAGERR_DETMODEL_I']**2 )

In [ ]:
dperpcut = ((ri_sdss - gr_sdss/8. < 0.55) & 
((gr_sdss>0.5)&(gr_sdss<0.96)|
 ((gr_sdss>1.43)&(gr_sdss<1.435))|
 ((gr_sdss>1.85)&(gr_sdss<3.0)) ) )

dperpcut_des = (ri_des - gr_des/8. < 0.5505) & (ri_des - gr_des/8. > 0.55035)
print np.sum(dperpcut), np.sum(dperpcut_des)

In [ ]:
from chainconsumer import ChainConsumer

#seed(0)
#cov = normal(size=(3, 3))
#data = multivariate_normal(normal(size=3), np.dot(cov, cov.T), size=100000)

data_gold = np.column_stack( [gr_gold, ri_gold] )
data_sdss = np.column_stack( [gr_sdss, ri_sdss] )
data_des = np.column_stack( [gr_des, ri_des] )

c = ChainConsumer()
c.add_chain(data_gold, parameters=["$g-r$", "$r-i$"])
#c.add_chain(data_des)
c.configure(plot_hists=False, cloud=True, sigmas=np.linspace(0, 3, 10), colors ='grey', 
            shade_alpha = 0.7, kde=True)


fig = c.plotter.plot()
ax = fig.axes[0]
ax.plot(gr_des, ri_des, 'g.', markersize = 1, alpha = 0.7)
#ax.plot(gr_sdss, ri_sdss, '.',markersize = 1, alpha = 0.7)
ax.plot(x, dperp, 'r-', linewidth=1)
ax.set_ylim(0,2)
ax.set_xlim(0, 3)

ax.text(2.0, 0.1, 'DES  Color')
plt.tight_layout()

#fig.savefig('../paper_figure/gri_des.pdf')

In [ ]:
c = ChainConsumer()

dat = data_gold.copy()
dat[:,0] = dat[:,0]+10
dat[:,1] = dat[:,1]+10
c.add_chain(dat, parameters=["$g-r$", "$r-i$"])
#c.add_chain(data_des)
c.configure(plot_hists=False, cloud=False, sigmas=np.linspace(0, 3, 10), colors ='grey', 
            shade_alpha = 0.5, kde=True)


fig = c.plotter.plot()
ax = fig.axes[0]
#ax.plot(gr_des, ri_des, 'g.', markersize = 1, alpha = 0.7)
ax.plot(gr_sdss, ri_sdss, '.',markersize = 1, alpha = 0.7)
ax.plot(x, dperp, 'r-', linewidth=1)
ax.set_ylim(0,2)
ax.set_xlim(0,3)

ax.text(1.8, 0.1, 'SDSS  Color')

plt.tight_layout()
#fig.savefig('../paper_figure/gri_sdss.pdf')

In [ ]:
from run_DMASS import *

cutmask = priorCut_test(gold_st82)
gold_cut_outlier = gold_st82[cutmask]

In [ ]:
#dperp_des = ri_des - gr_des/8.

gr_gold = gold_cut_outlier['MAG_DETMODEL_G']- gold_cut_outlier['MAG_DETMODEL_R']
ri_gold = gold_cut_outlier['MAG_DETMODEL_R']- gold_cut_outlier['MAG_DETMODEL_I']
dperp_gold = ri_gold - gr_gold/8.

In [ ]:
Ncmass = []
Nall = []
for dp in np.linspace(0.3, 0.6, 31):
    dpmask = dperp_gold > dp
    gold_train, cmass_train = matchCatalogsbyPosition(gold_cut_outlier[dpmask], cmass_stripe)
    Ncmass.append(gold_train.size)
    Nall.append(gold_cut_outlier[dpmask].size)
    print dp, gold_train.size, gold_cut_outlier[dpmask].size

In [ ]:
ncmass = np.array(Ncmass) * 1./np.array(Nall)
Nno = np.array(Nall) - np.array(Ncmass)

In [ ]:
fig, ax = plt.subplots()
ax.plot( np.linspace(0.3, 0.6, 31), ncmass*1./ncmass.max() )
ax.plot( np.linspace(0.3, 0.6, 31), Nno *1./np.array(Nall)  )
ax.axvline(x = 0.55, ls='--', color='k')

In [ ]:
def mixing_SDSS_color(data, suffix = '', sdss = None, cmass = None ):
    
    #filter = ['G', 'R', 'I', 'Z']
    filter = [1,2,3,4]
    mag = ['CMODELMAG', 'MODELMAG']
    magtag = mag#[ m+'_'+f+suffix for m in mag for f in filter ]
    #del magtag[0], magtag[2]
    err = [ 'CMODELMAGERR','MODELMAGERR']
    errtag = err#[ e+'_'+f for e in err for f in filter ]
    #del errtag[0], errtag[2]
    
    
    print data['CMODELMAG'][:,0].size
    
    X = [ data[mt][:,i] for mt in magtag for i in filter ]
    del X[0], X[2]
    Xerr = [ data[mt][:,i] for mt in errtag for i in filter]
    del Xerr[0], Xerr[2]
    #reddeningtag = 'XCORR_SFD98'

    X = np.vstack(X).T
    Xerr = np.vstack(Xerr).T
    # mixing matrix
    W = np.array([
                  [1, 0, 0, 0, 0, 0],
                  [0, 1, 0, 0, 0, 0],    # i cmag
                  [0, 0, 1, -1, 0, 0],   # g-r
                  [0, 0, 0, 1, -1, 0],   # r-i
                  [0, 0, 0, 0, 1, -1]])  # i-z

    X = np.dot(X, W.T)

    Xcov = np.zeros(Xerr.shape + Xerr.shape[-1:])
    Xcov[:, range(Xerr.shape[1]), range(Xerr.shape[1])] = Xerr**2
    Xcov = np.tensordot(np.dot(Xcov, W.T), W, (-2, -1))
    return X, Xcov

In [ ]:
X_cmass_train, Xcov_cmass_train = mixing_SDSS_color(cmass_train)

In [ ]:
X_cmass_train_new = [np.random.multivariate_normal( X_cmass_train[i], Xcov_cmass_train[i], 1 ) for i in range(cmass_train.size) ]

In [ ]:
X_cmass_train_new = np.array(X_cmass_train_new)

In [ ]:
new_gr_sdss = X_cmass_train_new[:,0][:,2]
new_ri_sdss = X_cmass_train_new[:,0][:,3]

In [ ]:
xbins = np.linspace(0, 3.0, 150)
ybins = np.linspace(0.2, 1.8, 120)
Ngal_sdss, xedges, yedges = np.histogram2d(gr_sdss, ri_sdss, bins=(xbins, ybins), normed=False)
Ngal_des, xedges, yedges = np.histogram2d(gr_des, ri_des, bins=(xbins, ybins), normed=False)

Ngal_sdss[Ngal_sdss == 0] = np.nan
Ngal_des[Ngal_des == 0] = np.nan

In [ ]:
from chainconsumer import ChainConsumer

c = ChainConsumer()

dat = data_gold.copy()
dat[:,0] = dat[:,0]+10
dat[:,1] = dat[:,1]+10
c.add_chain(dat, parameters=["$g-r$", "$r-i$"])
#c.add_chain(data_des)
c.configure(plot_hists=False, cloud=False, sigmas=np.linspace(0, 3, 10), colors ='grey', 
            shade_alpha = 0.5, kde=False)


fig = c.plotter.plot(figsize=(4,3.2))
ax = fig.axes[0]

im = ax.imshow(np.rot90(Ngal_sdss), extent=(np.amin(xbins), np.amax(xbins), np.amin(ybins), np.amax(ybins)),
        cmap=plt.cm.jet, aspect='auto', zorder = 2 )#, 
cbar = fig.colorbar(im, ax=ax)
cbar.set_label(r'$N_{\rm gal}$', fontsize=15)
ax.plot(x, dperp, 'r-', linewidth=1, zorder = 3)

#ax.errorbar( gr_sdss[dperpcut], ri_sdss[dperpcut], xerr = gr_sdss_err[dperpcut], 
#            yerr = ri_sdss_err[dperpcut], fmt = 'o', color='black', zorder=10)


ax.set_xlabel('g-r', fontsize=15)
ax.set_ylabel('r-i', fontsize=15)
ax.set_xlim(0,3.)
ax.set_ylim(0.2,1.8)

ax.text(1.9, 0.3, 'SDSS  Color' )
plt.tight_layout()

fig.savefig('../paper_figure/gri_sdss_color_cbar.pdf')

In [ ]:
from chainconsumer import ChainConsumer

#seed(0)
#cov = normal(size=(3, 3))
#data = multivariate_normal(normal(size=3), np.dot(cov, cov.T), size=100000)

data_gold = np.column_stack( [gr_gold, ri_gold] )
data_sdss = np.column_stack( [gr_sdss, ri_sdss] )
data_des = np.column_stack( [gr_des, ri_des] )

c = ChainConsumer()
c.add_chain(data_gold, parameters=["$g-r$", "$r-i$"])
#c.add_chain(data_des)
c.configure(plot_hists=False, cloud=True, sigmas=np.linspace(0, 3, 10), colors ='grey', 
            shade_alpha = 0.7, kde=False)


fig = c.plotter.plot(figsize=(4,3.2))
ax = fig.axes[0]

ax.imshow(np.rot90(Ngal_des), extent=(np.amin(xbins), np.amax(xbins), np.amin(ybins), np.amax(ybins)),
        cmap=plt.cm.jet, aspect='auto', zorder = 2, alpha = 1.0 )#, 
cbar = fig.colorbar(im, ax=ax)
cbar.set_label(r'$N_{\rm gal}$', fontsize=15)
ax.plot(x, dperp, 'r-', linewidth=1, zorder = 3)

#ax.errorbar( gr_sdss[dperpcut], ri_sdss[dperpcut], xerr = gr_sdss_err[dperpcut], 
#            yerr = ri_sdss_err[dperpcut], fmt = 'o', color='black', zorder=10)


ax.set_xlabel('g-r', fontsize=15)
ax.set_ylabel('r-i', fontsize=15)
ax.set_xlim(0,3.)
ax.set_ylim(0.2,1.8)

ax.text(2.0, 0.3, 'DES  Color')
plt.tight_layout()

fig.savefig('../paper_figure/gri_des_color_cbar.pdf')